In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import streamlit as st
import altair as alt

pd.set_option('display.max_columns', None)

In [ ]:
usecols = ['market','player_name','datagolf_base_history_fit','draftkings','fanduel','betmgm']

In [ ]:
# get american odds

win_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=win&odds_format=american&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
win = round(pd.read_csv(win_path, usecols=usecols),2)

top5_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=top_5&odds_format=american&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
top5 = round(pd.read_csv(top5_path, usecols=usecols),2)

top10_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=top_10&odds_format=american&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
top10 = round(pd.read_csv(top10_path, usecols=usecols),2)

top20_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=top_20&odds_format=american&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
top20 = round(pd.read_csv(top20_path, usecols=usecols),2)

# mc_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=make_cut&odds_format=american&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
# make_cut = round(pd.read_csv(mc_path, usecols=usecols),2)

american_odds = pd.concat([win,top5,top10,top20])
american_odds = american_odds[['market','player_name','datagolf_base_history_fit','fanduel','draftkings','betmgm']].convert_dtypes()

In [ ]:
# get decimal odds - make expected values

win_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=win&odds_format=percent&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
win = round(pd.read_csv(win_path, usecols=usecols),3)

top5_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=top_5&odds_format=percent&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
top5 = round(pd.read_csv(top5_path, usecols=usecols),3)

top10_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=top_10&odds_format=percent&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
top10 = round(pd.read_csv(top10_path, usecols=usecols),3)

top20_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=top_20&odds_format=percent&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
top20 = round(pd.read_csv(top20_path, usecols=usecols),3)

dec_odds = pd.concat([win,top5,top10,top20])
dec_odds = dec_odds[['market','player_name','datagolf_base_history_fit','fanduel','draftkings','betmgm']]

dec_odds['fanduel_ev'] = round(dec_odds['datagolf_base_history_fit'] * (1/dec_odds['fanduel']) - 1,2)
dec_odds['draftkings_ev'] = round(dec_odds['datagolf_base_history_fit'] * (1/dec_odds['draftkings']) - 1,2)
dec_odds['betmgm_ev'] = round(dec_odds['datagolf_base_history_fit'] * (1/dec_odds['betmgm']) - 1,2)

ev = dec_odds[['market','player_name','fanduel_ev','draftkings_ev','betmgm_ev']]

In [ ]:
# merge

odds = pd.merge(american_odds,ev,how='left',on=['market','player_name']).round(2)
odds = odds[['market','player_name','datagolf_base_history_fit','fanduel','fanduel_ev','draftkings','draftkings_ev','betmgm','betmgm_ev']].dropna()
odds.rename(columns={'datagolf_base_history_fit':'real_odds'}, inplace=True)

def plus_prefix(a):
    if a > 0:
        b = f"+{a}"
    else:
        b = a
    return b

odds['real_odds'] = odds['real_odds'].apply(plus_prefix)
odds['fanduel'] = odds['fanduel'].apply(plus_prefix)
odds['draftkings'] = odds['draftkings'].apply(plus_prefix)
odds['betmgm'] = odds['betmgm'].apply(plus_prefix)

odds

In [ ]:
odds_dict = {
    'win':'Win',
    'top_5':'Top 5',
    'top_10':'Top 10',
    'top_20':'Top 20'
}
odds['market'] = odds['market'].map(odds_dict)

In [ ]:
styled_odds = odds.style.background_gradient(cmap="bone", subset=['fanduel_ev','draftkings_ev','betmgm_ev']).format(precision=2)

In [ ]:
odds.style\
    .background_gradient(cmap="bone", subset=['fanduel_ev','draftkings_ev','betmgm_ev'])\
    .background_gradient(cmap="bone", subset=['draftkings_ev'])\
    .background_gradient(cmap="bone", subset=['betmgm_ev']).format(precision=2)

In [ ]:
def plus_prefix(a):
    if a > 0:
        b = f"+{a}"
    else:
        b = a
    return b

In [ ]:
odds

In [ ]:
def fix_names(df):
    """
    Takes in live datagolf scoring, cleans names, outputs list of active players this week
    """
    names = df['player_name'].str.split(expand=True)                  
    names[0] = names[0].str.rstrip(",")
    names[1] = names[1].str.rstrip(",")
    names['player'] = names[1] + " " + names[0]

    return names.player

odds['player_name'] = fix_names(odds)
odds

In [ ]:
pd.melt(odds_master,id_vars=['player_name','market'],value_vars=['datagolf_baseline', 'fanduel', 'fanduel_val',
       'draftkings', 'draftkings_val', 'betmgm', 'betmgm_val', 'caesars',
       'caesars_val'])

In [ ]:
odds_master = odds[['market','player_name','datagolf_base_history_fit','fanduel','fanduel_val','draftkings','draftkings_val','betmgm','betmgm_val']].dropna()#,'caesars','caesars_val']]
odds_master.columns = ['Market','Player','Odds','FD Odds','FD EV','DK Odds','DK EV','BetMGM Odds','MGM EV']#,'Caesars','Caesars EV']


odds_dict = {
    'win':'Win',
    'top_5':'Top 5',
    'top_10':'Top 10',
    'top_20':'Top 20'
}
odds_master['Market'] = odds_master['Market'].map(odds_dict)

In [ ]:
odds = pd.merge(american_odds,ev,how='left',on=['market','player_name']).round(2)
odds = odds[['market','player_name','datagolf_base_history_fit','fanduel','fanduel_ev','draftkings','draftkings_ev','betmgm','betmgm_ev']].dropna()
odds.rename(columns={'datagolf_base_history_fit':'real_odds'}, inplace=True)
odds

In [68]:
import pandas as pd
import plotly.express as px
import numpy as np
import streamlit as st
import altair as alt

pd.set_option('display.max_columns', None)

dg_key = "e297e933c3ad47d71ec1626c299e"

def fetch_odds(market):
    odds_format = 'american'
    odds = {}
    for market_type in ['win', 'top_5', 'top_10', 'top_20']:
        url = f"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market={market_type}&odds_format={odds_format}&file_format=csv&key={dg_key}"
        usecols = ['market', 'player_name', 'datagolf_base_history_fit', 'draftkings', 'fanduel', 'betmgm']
        data = pd.read_csv(url, usecols=usecols)
        odds[market_type] = round(data, 2)
    return odds[market]

def calculate_ev(odds):
    odds['fanduel_ev'] = round(odds['real_odds'] * (1 / odds['fanduel']) - 1, 2)
    odds['draftkings_ev'] = round(odds['real_odds'] * (1 / odds['draftkings']) - 1, 2)
    odds['betmgm_ev'] = round(odds['real_odds'] * (1 / odds['betmgm']) - 1, 2)
    return odds#[['fanduel_ev', 'draftkings_ev', 'betmgm_ev']]

def plus_prefix(a):
    if a > 0:
        return f"+{a}"
    return a

def fix_names(df):
    """
    Takes in live datagolf scoring, cleans names, outputs list of active players this week
    """
    names = df['player_name'].str.split(expand=True)                  
    names[0] = names[0].str.rstrip(",")
    names[1] = names[1].str.rstrip(",")
    names['player'] = names[1] + " " + names[0]

    return names.player

In [72]:
odds = fetch_odds('win')
odds = pd.concat([odds, fetch_odds('top_5'), fetch_odds('top_10'), fetch_odds('top_20')]).dropna()

odds.rename(columns={'datagolf_base_history_fit':'real_odds'},inplace=True)
odds['player_name'] = fix_names(odds)

odds = calculate_ev(odds)

In [74]:
odds['market'] = odds['market'].replace({'win': 'Win', 'top_5': 'Top 5', 'top_10': 'Top 10', 'top_20': 'Top 20'})
odds

,fanduel,player_name,betmgm,draftkings,real_odds,market,fanduel_ev,draftkings_ev,betmgm_ev
0,450.0,Scottie Scheffler,400,450,584.0,Win,0.30,0.30,0.46
1,1100.0,Rory McIlroy,1100,1100,1348.0,Win,0.23,0.23,0.23
2,1400.0,Xander Schauffele,1400,1400,1362.0,Win,-0.03,-0.03,-0.03
3,1100.0,Jon Rahm,1200,1100,1600.0,Win,0.45,0.45,0.33
4,2000.0,Hideki Matsuyama,1800,2000,2530.0,Win,0.27,0.27,0.41
...,...,...,...,...,...,...,...,...,...
84,3500.0,Mike Weir,3500,3500,27219.0,Top 20,6.78,6.78,6.78
85,20000.0,Jose Olazabal,5000,20000,890622.0,Top 20,43.53,43.53,177.12
86,4000.0,Vijay Singh,2800,4000,24295.0,Top 20,5.07,5.07,7.68
87,11000.0,Jasper Stubbs,2000,11000,559256.0,Top 20,49.84,49.84,278.63


In [75]:

odds = fetch_odds('win')
odds = pd.concat([odds, fetch_odds('top_5'), fetch_odds('top_10'), fetch_odds('top_20')]).dropna()
odds.rename(columns={'datagolf_base_history_fit':'real_odds'},inplace=True)
odds['player_name'] = fix_names(odds)

odds = calculate_ev(odds)

odds['market'] = odds['market'].replace({'win': 'Win', 'top_5': 'Top 5', 'top_10': 'Top 10', 'top_20': 'Top 20'})

odds = odds[['market', 'player_name', 'real_odds', 'fanduel','fanduel_ev', 'draftkings','draftkings_ev', 'betmgm','betmgm_ev']].convert_dtypes()

odds['real_odds'] = odds['real_odds'].apply(plus_prefix)
odds['fanduel'] = odds['fanduel'].apply(plus_prefix)
odds['draftkings'] = odds['draftkings'].apply(plus_prefix)
odds['betmgm'] = odds['betmgm'].apply(plus_prefix)

odds['market'] = odds['market'].replace({'win': 'Win', 'top_5': 'Top 5', 'top_10': 'Top 10', 'top_20': 'Top 20'})

odds.columns = ['Market','Player','True Odds','FD','FD EV','DK','DK EV','MGM','MGM EV']
odds

,Market,Player,True Odds,FD,FD EV,DK,DK EV,MGM,MGM EV
0,Win,Scottie Scheffler,+584,+450,0.3,+450,0.3,+400,0.46
1,Win,Rory McIlroy,+1348,+1100,0.23,+1100,0.23,+1100,0.23
2,Win,Xander Schauffele,+1362,+1400,-0.03,+1400,-0.03,+1400,-0.03
3,Win,Jon Rahm,+1600,+1100,0.45,+1100,0.45,+1200,0.33
4,Win,Hideki Matsuyama,+2530,+2000,0.27,+2000,0.27,+1800,0.41
...,...,...,...,...,...,...,...,...,...
84,Top 20,Mike Weir,+27219,+3500,6.78,+3500,6.78,+3500,6.78
85,Top 20,Jose Olazabal,+890622,+20000,43.53,+20000,43.53,+5000,177.12
86,Top 20,Vijay Singh,+24295,+4000,5.07,+4000,5.07,+2800,7.68
87,Top 20,Jasper Stubbs,+559256,+11000,49.84,+11000,49.84,+2000,278.63


In [80]:
odds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 348 entries, 0 to 88
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Market     348 non-null    string 
 1   Player     348 non-null    string 
 2   True Odds  348 non-null    object 
 3   FD         348 non-null    object 
 4   FD EV      348 non-null    Float64
 5   DK         348 non-null    object 
 6   DK EV      348 non-null    Float64
 7   MGM        348 non-null    object 
 8   MGM EV     348 non-null    Float64
dtypes: Float64(3), object(4), string(2)
memory usage: 36.3+ KB


In [78]:
odds.style.background_gradient(cmap="gist_heat")#, subset=['FD EV', 'DK EV', 'MGM EV']).format(precision=2)

KeyError: '`Styler.apply` and `.applymap` are not compatible with non-unique index or columns.'

In [66]:
styled_odds = odds.style.background_gradient(cmap="gist_heat", subset=['FD EV', 'DK EV', 'MGM EV']).format(precision=2)
styled_odds

KeyError: '`Styler.apply` and `.applymap` are not compatible with non-unique index or columns.'